# Authorization & securitySchemes

OAS allows to specify authorization policies in the spec,
under `components.securitySchemes`.

Between supported security schemes we have:

- oauth and oidc
- basic auth
- JWT
- mutualTLS ( in OAS 3.1 )

**When passing credentials in HTTP headers or payload you MUST use TLS**

`connexion` can reference a python function via `x-basicInfoFunc` 


### basic auth

Here we are defining the `myBasicAuth` security scheme.

```
components:
 securitySchemes:
   myBasicAuth:
     type: http
     scheme: basic
     x-basicInfoFunc: security.my_auth
```

We can then reference `myBasicAuth` in one or more paths

```
paths:
  /echo
    get:
      security:
      - myBasicAuth: []
      ...
      operationId: api.get_echo
      ...
```

### Exercise: add securitySchemes

Modify the OAS3 spec in [ex-06-01-auth.yaml](/edit/notebooks/oas3/ex-06-01-auth-ok.yaml) and:

- add a `myBasicAuth` security schemes like the above;
- reference `myBasicAuth` in `get /echo` path;
- validate the spec in your swagger editor and 
  check what changed in the swagger-ui

### Implement my_auth

Implement the `my_auth` function in [security.py](/edit/notebooks/oas3/security.py) so that:

- when username==password the user is authenticated

Use the cell below to implement it, 

In [2]:
# Test here the my_auth implementation.
def my_auth(username, password,required_scopes=None):
    """An dummy authentication function.
       :params: username, the username
       :params: password, the password
       :params: scopes, the scope
       :returns: `{"sub": username, "scope": ""}` on success, 
                 None on failure
    """   
    raise NotImplementedError


### Test the my_auth implementation

Run the spec in the [terminal](/terminals/1)
with the usual

```
connexion run /node/notebooks/oas3/ex-06-01-auth-ok.yaml
```

play a bit with the [Swagger UI](https://TODO)

Try to curl some requests!


In [3]:
!curl http://localhost:5000/datetime/v1/echo -kv

*   Trying 127.0.0.1...
* TCP_NODELAY set
* Connected to localhost (127.0.0.1) port 5000 (#0)





* HTTP 1.0, assume close after body






{
  "detail": "No authorization token provided",
  "status": 401,
  "title": "Unauthorized",
  "type": "about:blank"
}
* Curl_http_done: called premature == 0
* Closing connection 0


### Bearer token & JWT Security

Bearer tokens are supported:

- in OAS via the `scheme: bearer`
- in connexion via the `x-bearerInfoFunc`

```
components:
  securitySchemes:
    jwt:
      type: http
      scheme: bearer
      bearerFormat: JWT
      x-bearerInfoFunc: security.decode_token

```

Once you send the header

`Authorization: Bearer token`

the `token` string will be passed to a function like the following

NOTE: the `bearerFormat` is a free identifier of the token format and the associated syntax may not be enforced by the spec.

In [10]:

def decode_token(token):
    """
        :param: token, a generic token
        :return: a json object compliant with RFC7662 OAUTH2
    """
    
    if token_is_valid:
        return {
             'iss': 'http://api.example.com',
             'iat': 1561296569,
             'exp': 1561296571,
             'sub': 'ioggstream',
             'scope': ['local']
            }

In [ ]:
from random import randint


def get_status():
    p = randint(1, 5)
    if p == 5:
        return problem(
            status=503,
            title="Service Temporarily Unavailable",
            detail="Retry after the number of seconds specified in the the Retry-After header.",
            headers={'Retry-After': str(p)}
        )
    return problem(
        status=200,
        title="OK",
        detail="So far so good."
    )

You can learn more on that on:

- https://github.com/zalando/connexion/blob/master/docs/security.rst
        